In [11]:
# Bu kütüphaneyi rastgele sayı üretmek için kullanabilirsiniz
import random

# Savaş Alanı

* Savaş alanının boyutunu (satır ve sütün sayısını biz belirliyoruz)
* Savaş alanının güncel görüntüsünü her el ekrana yazdırabilmek için \_\_str\_\_ metodu tanımlayın
* SavasAlani class'ında hem kendimizin savaş alanını hem de bilgisayarın savaş alanını tanımlıyoruz
* Kendi savaş alanımızdaki gemilerin konumlarını gösterirken, bilgisayarın savaş alanındaki gemilerin konumlarını bilmediğimiz için, bunları gösteremiyoruz
* Savaş alanına yerleştirmek istenilen gemilerin koordinatlarını (ve yatay mı dıkey mı olduğunu) alan ve bu gemileri savaş alanını yerleştiren bir metod bulunuyor
* Her el, hem kendi hemde bilgisayarın savaş alanını atış yapılan noktalara göre güncelleyen bir metod bulunuyor

In [22]:
def renkli_print(print_str, renk):
    renkler = {"kirmizi": 31, "yesil": 32, "sari": 33}
    renk_kodu = renkler[renk]
    return f"\033[1;{renk_kodu}m{print_str}\033[0m"


class SavasAlani:
    def __init__(self, n_satir, n_sutun):
        self.n_satir = n_satir
        self.n_sutun = n_sutun
        self.gemi_sekli = "O"
        self.savas_alani = self.init_savas_alani()
        self.diger_savas_alani = self.init_savas_alani()
        self.savas_alanlari = {
            "kendi": self.savas_alani,
            "diger": self.diger_savas_alani
        }
        self.guncelleme_kodlari = {
            "karavana": renkli_print("x", "sari"),
            "isabet": renkli_print("x", "yesil"),
            "hasar": renkli_print("x", "kirmizi")
        }
        
    def init_savas_alani(self):
        matris = []
        for satir in range(self.n_satir):
            matris.append(["-"]*self.n_sutun)
        return matris
    
    def gemi_yerlestir(self, gemi, satir_numarasi, sutun_numarasi, yatay_dikey):
        for i in range(gemi): # gemi = 4
            if yatay_dikey == "yatay":
                self.savas_alani[satir_numarasi][sutun_numarasi+i] = self.gemi_sekli
            else:
                self.savas_alani[satir_numarasi+i][sutun_numarasi] = self.gemi_sekli
                
    def girilen_koordinatlar_uygun_mu(self, gemi, satir_numarasi, sutun_numarasi, yatay_dikey):
        konum = sutun_numarasi if yatay_dikey == "yatay" else satir_numarasi
        kenar = self.n_sutun if yatay_dikey == "yatay" else self.n_satir
        alan_yeterli_mi = self.alan_yeterli_mi(gemi, konum, kenar)
        
        if not alan_yeterli_mi:
            raise RuntimeError
        
        pozisyon_uygun_mu= self.pozisyon_uygun_mu(gemi, satir_numarasi, sutun_numarasi, yatay_dikey)
        
        if not pozisyon_uygun_mu:
            raise RuntimeError
        
    def alan_yeterli_mi(self, gemi, konum, kenar):
        if gemi + konum >= kenar:
            return False
        return True
    
    def pozisyon_uygun_mu(self, gemi, satir_numarasi, sutun_numarasi, yatay_dikey):
        for i in range(gemi):
            if yatay_dikey == "yatay":
                sembol = self.savas_alani[satir_numarasi][sutun_numarasi+i]
            else:
                sembol = self.savas_alani[satir_numarasi+i][sutun_numarasi]
            if sembol == self.gemi_sekli:
                return False
        return True
    
    def guncelle(self, satir_numarasi, sutun_numarasi, bilgisayar_savas_alani):
        if bilgisayar_savas_alani.savas_alani[satir_numarasi][sutun_numarasi] == self.gemi_sekli:
            self._guncelle("diger", satir_numarasi, sutun_numarasi, "isabet")
            bilgisayar_savas_alani._guncelle("kendi", satir_numarasi, sutun_numarasi, "hasar")
        else:
            self._guncelle("diger", satir_numarasi, sutun_numarasi, "karavana")
            bilgisayar_savas_alani._guncelle("kendi", satir_numarasi, sutun_numarasi, "karavana")
            
    def _guncelle(self, hangi, satir_numarasi, sutun_numarasi, sonuc):
        self.savas_alanlari[hangi][satir_numarasi][sutun_numarasi] = self.guncelleme_kodlari[sonuc]
            
    
    def __str__(self):
        kendi_savas_alanimiz = self.savas_alani_olustur(self.savas_alani)
        diger_savas_alani = self.savas_alani_olustur(self.diger_savas_alani)
        
        kendi_savas_alanimiz_liste = kendi_savas_alanimiz.split("\n")
        diger_savas_alani_liste = diger_savas_alani.split("\n")
        fazla_karakter = len(str(self.n_sutun)) - 1
        diger_savas_alani_liste[0] = diger_savas_alani_liste[0][fazla_karakter:]

        savas_alani_str = [(10*" ").join([kendi, diger])
                           for kendi, diger in zip(kendi_savas_alanimiz_liste, diger_savas_alani_liste)]
        return "\n".join(savas_alani_str)
        
    def savas_alani_olustur(self, savas_alani):
        satir_numaralari = list(range(self.n_satir))
        satir_numaralari = list(map(str, satir_numaralari))
            
        sutun_numaralari = list(range(self.n_sutun))
        sutun_numaralari = list(map(str, sutun_numaralari))
        
        max_satir_numarasi = len(satir_numaralari[-1]) + 1
        max_sutun_numarasi = len(sutun_numaralari[-1])
        
        bosluk_str = " " * max_satir_numarasi
        savas_alani_str = bosluk_str
        for sutun_numarasi in sutun_numaralari: # ["0", 1", "2", "3",...,  "10"]
            bosluk_sayisi = max_sutun_numarasi - len(sutun_numarasi) + 1
            bosluk_str = " " * bosluk_sayisi
            savas_alani_str += sutun_numarasi + bosluk_str
        savas_alani_str = savas_alani_str[:-len(bosluk_str)]
        savas_alani_str += "\n"
        
        for satir_numarasi in satir_numaralari: # ["0", 1", "2", "3"....]
            bosluk_sayisi = max_satir_numarasi - len(satir_numarasi)
            bosluk_str = " " * bosluk_sayisi
            
            satir = savas_alani[int(satir_numarasi)]
            satir_str = (max_sutun_numarasi*" ").join(satir)
            
            savas_alani_str += satir_numarasi + bosluk_str + satir_str + "\n" 
        return savas_alani_str

# Savaş Gemileri
* Bu class oyuncudan savaş gemilerini yerleştirmesini istediği koordinatları alıyor ve bu koordinatları SavasAlanı classına gönderiyor (oyuncu, bu koordinatların rastgele seçilmesini de isteyebilir)
* Aynı zamanda bütün gemilerin batıp batmadığını kontrol eden bir metodu bulunuyor

In [23]:
class SavasGemileri:
    def __init__(self, gemiler, savas_alani):
        self.gemiler = gemiler # [3, 5, 6, 3, 4]
        self.savas_alani = savas_alani
    
    def gemilerin_koordinatlarini_al(self, rasgele=False, bilgisayar=False):
        for gemi in self.gemiler:
            satir_numarasi = None
            sutun_numarasi = None
            yatay_dikey = None
            while satir_numarasi is None or sutun_numarasi is None or yatay_dikey is None:
                try:
                    if rasgele:
                        girilen_degerler = self.rasgele_konum_yarat()
                    else:
                        girilen_degerler = self.oyuncudan_girilen_degerleri_al(gemi)
                    satir_numarasi, sutun_numarasi, yatay_dikey = girilen_degerler.split(" ")
                    if yatay_dikey not in ["yatay", "dikey"]:
                        raise RuntimeError
                    satir_numarasi = int(satir_numarasi)
                    sutun_numarasi = int(sutun_numarasi)
                    self.savas_alani.girilen_koordinatlar_uygun_mu(gemi, satir_numarasi, 
                                                                   sutun_numarasi, yatay_dikey)
                except RuntimeError:
                    if not rasgele:
                        print("Yanlis degerler girdiniz, lutfen bilgi mesajini dikkatlice okuyunuz")
                    satir_numarasi = None
                    # sutun_numarasi = None
                    # yatay_dikey = None 
                else:
                    self.savas_alani.gemi_yerlestir(gemi, satir_numarasi, sutun_numarasi, yatay_dikey)
        if not bilgisayar:
            print(self.savas_alani)
            
    def oyuncudan_girilen_degerleri_al(self, gemi):
        girilen_degerler = input(f"""
Lutfen aralarinda bosluk birakarak {gemi} boyutundaki geminizi 
savas alaninda nereye koymak istediginizi girin. 
Girilen degerler sirasiyla: satir_numarasi (int), sutun_numarasi (int), yatay yada dikey (str)
Savas alaninizin su anki hali:
{self.savas_alani}
""")
        return girilen_degerler
    
    def rasgele_konum_yarat(self):
        satir_numarasi = random.randint(0, self.savas_alani.n_satir-1)
        sutun_numarasi = random.randint(0, self.savas_alani.n_sutun-1)
        yatay_dikey = random.choice(["yatay", "dikey"])
        girilen_degerler = f"{satir_numarasi} {sutun_numarasi} {yatay_dikey}"
        return girilen_degerler
        

# Oyuncu

* Oyuncu classı, her el oyuncudan atış yapmak istediği noktaların koordınatlarını alıyor, ve savaş alanını bu koordinatlara göre güncelliyor
* Aynı zamanda bilgisayar oyuncusunun rastgele atışlar yapmasını sağlıyor

In [24]:
class Oyuncu:
    def __init__(self, savas_alani, savas_gemileri, bilgisayar_mi):
        self.savas_alani = savas_alani
        self.savas_gemileri = savas_gemileri
        self.bilgisayar_mi = bilgisayar_mi
        
    def oyna(self, bilgisayar):
        for i in range(3):
            satir_numarasi = None
            sutun_numarasi = None
            while satir_numarasi is None or sutun_numarasi is None:
                try:
                    if self.bilgisayar_mi:
                        girilen_degerler = self.rasgele_konum_yarat()
                    else:
                        girilen_degerler = self.oyuncudan_girilen_degerleri_al()
                    satir_numarasi, sutun_numarasi = girilen_degerler.split(" ")
                    satir_numarasi = int(satir_numarasi)
                    sutun_numarasi = int(sutun_numarasi)
                    
                except RuntimeError:
                    if not rasgele:
                        print("Yanlis degerler girdiniz, lutfen bilgi mesajini dikkatlice okuyunuz")
                    satir_numarasi = None
                else:
                    self.savas_alani.guncelle(satir_numarasi, sutun_numarasi, bilgisayar.savas_alani)
        print(self.savas_alani)
        
    def oyuncudan_girilen_degerleri_al(self):
        girilen_degerler = input(f"Lutfen atis yapmak istediginiz satir ve sutun koordinatlarini giriniz: ")
        return girilen_degerler

    def rasgele_konum_yarat(self):
        satir_numarasi = random.randint(0, self.savas_alani.n_satir-1)
        sutun_numarasi = random.randint(0, self.savas_alani.n_sutun-1)
        girilen_degerler = f"{satir_numarasi} {sutun_numarasi}"
        return girilen_degerler

In [25]:
savas_alani = SavasAlani(15, 15)
bilgisayar_savas_alani = SavasAlani(15, 15)

gemiler = [3, 4, 5]

savas_gemileri = SavasGemileri(gemiler, savas_alani)
savas_gemileri.gemilerin_koordinatlarini_al(rasgele=True, bilgisayar=False)

bilgisayar_savas_gemileri = SavasGemileri(gemiler, bilgisayar_savas_alani)
bilgisayar_savas_gemileri.gemilerin_koordinatlarini_al(rasgele=True, bilgisayar=True)

oyuncu = Oyuncu(savas_alani, savas_gemileri, False)
bilgisayar = Oyuncu(bilgisayar_savas_alani, bilgisayar_savas_gemileri, True)

oyuncu.oyna(bilgisayar)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14            0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
0  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -          0  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
1  -  -  -  -  -  -  -  -  O  -  -  -  -  -  -          1  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
2  -  O  O  O  O  -  -  -  O  -  -  -  -  -  -          2  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
3  -  -  -  -  -  -  -  -  O  -  -  -  -  -  -          3  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
4  -  -  -  -  -  -  -  -  O  -  -  O  O  O  -          4  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
5  -  -  -  -  -  -  -  -  O  -  -  -  -  -  -          5  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -          6  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -          7  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
8  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -          8  -  -  -  -  -

# Amiral Battı
* Bu class diğer yaratılan bütün classları kullanarak oyunun akışını yönetiyor
* Görevlerinden bir tanesi ise rastgele savaş gemileri yaratmak (boyutlarını belirlemek, örn: 3, 5, 4

In [ ]:
class AmiralBatti:
    def __init__(self, n_satir, n_sutun, n_gemi):
        oyuncu_savas_alani = SavasAlani(n_satir, n_sutun)
        bilgisayar_savas_alani = SavasAlani(n_satir, n_sutun)
        
        # Gemileri yarat
        self.gemiler = []
        
        oyuncu_savas_gemileri = SavasGemileri(self.gemiler, oyuncu_savas_alani)
        bilgisayar_savas_gemileri = SavasGemileri(self.gemiler, bilgisayar_savas_alani)
        
        self.oyuncu = Oyuncu(oyuncu_savas_alani, oyuncu_savas_gemsileri, False)
        self.bilgisayar = Oyuncu(bilgisayar_savas_alani, bilgisayar_savas_gemileri, True)

In [ ]:
n_satir = 10
n_sutun = 10
n_gemi = 4

amiral_batti = AmiralBatti(n_satir, n_sutun, n_gemi)
amiral_batti.basla()